In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3
import shapely
from geopandas import GeoDataFrame,GeoSeries
import folium
import matplotlib
%matplotlib inline

/Users/rakeenahmed/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Automated Speed Enforcement

In [80]:
speed_enforcement = gpd.read_file('Automated Speed Enforcement Locations.geojson')

In [81]:
speed_enforcement.head()

,_id,Location_Code,Ward,Status,location,FID,geometry
0,1,A298,1 - Etobicoke North,Active,Thistle Down Blvd. East of Albion Rd.,1,MULTIPOINT (-79.56022 43.73667)
1,2,A318,1 - Etobicoke North,Active,Kipling Ave. North of Coolhurst Dr.,2,MULTIPOINT (-79.56942 43.71741)
2,3,A319,1 - Etobicoke North,Active,Mount Olive Dr. North of Dorward Dr.,3,MULTIPOINT (-79.59493 43.74885)
3,4,A252,2 - Etobicoke Centre,Active,Bloor St. W. Near 4034 Bloor St. W.,4,MULTIPOINT (-79.55458 43.63745)
4,5,A253,2 - Etobicoke Centre,Active,Allanhurst Dr. Near Mulham Place,5,MULTIPOINT (-79.51767 43.68003)


In [82]:
#convert column names to lowercase
speed_enforcement.columns = [s.strip().lower() for s in speed_enforcement.columns]

In [83]:
#set location code as index
speed_enforcement = speed_enforcement.set_index('location_code')

In [84]:
speed_enforcement = speed_enforcement.drop(columns=['_id','ward','status','location','fid'])

In [85]:
#adding a new column binary indicator for speed_enforcement
speed_enforcement['automatic_speed_enforcement'] = "Yes"

In [86]:
speed_enforcement.head()

,geometry,automatic_speed_enforcement
location_code,,
A298,MULTIPOINT (-79.56022 43.73667),Yes
A318,MULTIPOINT (-79.56942 43.71741),Yes
A319,MULTIPOINT (-79.59493 43.74885),Yes
A252,MULTIPOINT (-79.55458 43.63745),Yes
A253,MULTIPOINT (-79.51767 43.68003),Yes


In [87]:
#converting multipoing geometry to single point geometry
#speed_enforcement = speed_enforcement.explode()

In [88]:
#extract latitude and longitudes from single line geometry
#speed_enforcement['latitude'] = speed_enforcement.apply(lambda x: x.geometry.y, axis=1)
#speed_enforcement['longitude'] = speed_enforcement.apply(lambda x: x.geometry.x, axis=1)

In [89]:
#combining latitude and longitudes to form coordinates column
#speed_enforcement['coordinates'] = speed_enforcement.apply(lambda x: (x.latitude, x.longitude), axis=1)

#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    a = coordinates[0]
    b = coordinates[1]
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#speed_enforcement['hex_cell'] = speed_enforcement['coordinates'].apply(get_hex_cell)

IndentationError: unexpected indent (3092004022.py, line 13)

In [92]:
speed_enforcement.head()

,geometry,speed_enforcement_automatic_speed_enforcement
location_code,,
A298,MULTIPOINT (-79.56022 43.73667),Yes
A318,MULTIPOINT (-79.56942 43.71741),Yes
A319,MULTIPOINT (-79.59493 43.74885),Yes
A252,MULTIPOINT (-79.55458 43.63745),Yes
A253,MULTIPOINT (-79.51767 43.68003),Yes


In [93]:
#adding column prefix speed_enforcement to all columns
speed_enforcement.columns = ['speed_enforcement_' + s if s !='geometry' else s for s in speed_enforcement.columns] 

In [94]:
speed_enforcement.to_file('speed_enforcement_processed.geojson')

# Traffic Camera List

In [109]:
traffic_cameras = gpd.read_file('Traffic Camera List (CSV).csv')

In [110]:
type(traffic_cameras)

geopandas.geodataframe.GeoDataFrame

In [111]:
#converting column headers to lowercase
traffic_cameras.columns = [s.strip().lower().replace(' ','_') for s in traffic_cameras.columns]

In [112]:
traffic_cameras.head()

,camera_number,latitude,longitude,main_road,cross_street,group,traffic_image,north_reference_static_image,east_reference_static_image,south_reference_static_image,west_reference_static_image,geometry
0,Camera8000,43.64945,-79.371464,ABELL ST,BABY POINT RD,Arterial,http://opendata.toronto.ca/transportation/tmc...,,,,,None
1,Camera8001,43.643120,-79.381386,YORK ST,BREMNER BLVD / RAPTORS WAY,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,None
2,Camera8002,43.64222,-79.384068,LOWER SIMCOE ST,BREMNER BLVD,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,None
3,Camera8003,43.649461,-79.371267,JARVIS ST,FRONT ST E / LOWER JARVIS ST,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,None
4,Camera8004,43.645153,-79.382574,UNIVERSITY AVE,FRONT ST W / YORK ST,Arterial,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,http://opendata.toronto.ca/transportation/tmc...,None


In [113]:
#set index to camera number
traffic_cameras = traffic_cameras.set_index('camera_number')

In [114]:
#dropping unnecessary columns
traffic_cameras = traffic_cameras.drop(traffic_cameras.columns[3:-1],axis=1)

In [115]:
type(traffic_cameras)

geopandas.geodataframe.GeoDataFrame

In [116]:
traffic_cameras.head()

,latitude,longitude,main_road,geometry
camera_number,,,,
Camera8000,43.64945,-79.371464,ABELL ST,None
Camera8001,43.643120,-79.381386,YORK ST,None
Camera8002,43.64222,-79.384068,LOWER SIMCOE ST,None
Camera8003,43.649461,-79.371267,JARVIS ST,None
Camera8004,43.645153,-79.382574,UNIVERSITY AVE,None


In [117]:
#adding a geometry column to traffic cameras

traffic_cameras.crs = 'EPSG:3857'

traffic_cameras = traffic_cameras.to_crs('EPSG:3857')

geometries = gpd.points_from_xy(x=traffic_cameras.latitude,y=traffic_cameras.longitude,crs='EPSG:3857')

traffic_cameras['geometry'] = geometries.to_crs('EPSG:3857')

In [118]:
#adding a column for traffic_cameras = "Yes"
traffic_cameras['traffic_cameras'] = "Yes"

In [119]:
#combining latitude and longitudes to form coordinates column
#traffic_cameras['coordinates'] = traffic_cameras.apply(lambda x: (x.latitude, x.longitude), axis=1)

#create hex cells for bike lanes

#create coordinate column
#pedestrian_crossover_exploded['coordinates'] = pedestrian_crossover_exploded['latitude'].astype('str') + ',' + pedestrian_crossover_exploded['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
#def get_hex_cell(coordinates):
    #a = float(coordinates.split(',')[0])
    #b = float(coordinates.split(',')[1])
    #a = coordinates[0]
    #b = coordinates[1]
    #return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
#traffic_cameras['hex_cell'] = traffic_cameras['coordinates'].apply(get_hex_cell)

In [120]:
traffic_cameras.head()

,latitude,longitude,main_road,geometry,traffic_cameras
camera_number,,,,,
Camera8000,43.64945,-79.371464,ABELL ST,POINT (43.649 -79.371),Yes
Camera8001,43.643120,-79.381386,YORK ST,POINT (43.643 -79.381),Yes
Camera8002,43.64222,-79.384068,LOWER SIMCOE ST,POINT (43.642 -79.384),Yes
Camera8003,43.649461,-79.371267,JARVIS ST,POINT (43.649 -79.371),Yes
Camera8004,43.645153,-79.382574,UNIVERSITY AVE,POINT (43.645 -79.383),Yes


In [121]:
#adding column prefix speed_enforcement to all columns
traffic_cameras.columns = ['traffic_cameras_' + s if s!= 'geometry' else s for s in traffic_cameras.columns]

In [122]:
traffic_cameras.to_file('traffic_cameras_processed.geojson')